In [1]:
import numpy as np

import sys
from pathlib import Path

from scipy.io import savemat

# Add project paths
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Import experiment management from HDMF src folder
from simulation_runner import HDMFSimulationRunner
from experiment_manager import ExperimentManager
from plotting import HDMFResultsPlotter
from utils.data_loading import load_all_sc_matrices

experiment_manager = ExperimentManager(project_root, config_path="Homeostatic_Grid", results_dir="/network/iss/cohen/data/Ivan/fastHDMF")
res = experiment_manager.load_experiment_results()
plotter = HDMFResultsPlotter(experiment_manager)
config = experiment_manager.current_config

2025-09-22 15:44:56,142 - hdmf_experiment_Homeostatic_Grid - INFO - Experiment 'Homeostatic_Grid' initialized
2025-09-22 15:44:56,143 - hdmf_experiment_Homeostatic_Grid - INFO - Config: Homeostatic grid for objective rate 3.44. Used to then calculate LR->DECAY coefficients. Now supports custom parameter generation functions.
2025-09-22 15:44:56,144 - hdmf_experiment_Homeostatic_Grid - INFO - Results will be saved to: /network/iss/cohen/data/Ivan/fastHDMF/Homeostatic_Grid
2025-09-22 15:44:56,152 - hdmf_experiment_Homeostatic_Grid - INFO - Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Homeostatic_Grid.yaml


INFO: Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Homeostatic_Grid.yaml


In [2]:
hom_grid = np.stack(res['full_results']['observables']['mean_rates'].flatten())
hom_grid = np.reshape(hom_grid, (100,110,17,90))
hom_grid = 3.44 - np.mean(hom_grid, axis=-1)
hom_grid_mean = np.mean(hom_grid, axis=2)
hom_grid_std = np.std(hom_grid, axis=2)


In [18]:
# Parameter ranges from config (fallbacks if missing)


# Check explicit Homeostatic_Grid-like config where functions are used
def _generate_parameter_values(spec: dict) -> np.ndarray:
    """Generate parameter values from spec with optional custom function"""
    if "fun" in spec:
        fun_name = spec["fun"]
        args = spec.get("args", [])
        kwargs = spec.get("kwargs", {})
        
        try:
            func = eval(fun_name)
            values = func(*args, **kwargs)
            return np.array(values, dtype=float)
        except Exception as e:
            raise ValueError(f"Error calling function '{fun_name}': {e}")
    else:
        # Default: start, end, step
        values = np.arange(spec["start"], spec["end"], spec["step"], dtype=float)
        return np.round(values, 12)
    
over_cfg = config.get('simulation', {}).get('over', {})
G_range = _generate_parameter_values(config['simulation']['over']['G'])
LR_range = _generate_parameter_values(config['grid']['lrj'])
DECAY_range = _generate_parameter_values(config['grid']['taoj'])

nlr = len(LR_range)
ndec = len(DECAY_range)
nG = len(G_range)

# find g axis by size match
axes_sizes = hom_grid.shape
print("Assuming G Axis is the last axis")
hom_grid_mean = np.mean(hom_grid, axis=-1)
std_hom_grid = np.std(hom_grid, axis=-1)

min_mm_pos = np.argmin(np.abs(hom_grid_mean), axis=1)

x_idx = np.arange(nlr)
# Extract the decay values corresponding to the minimum mismatch for each learning rate
y = DECAY_range[min_mm_pos]
if config.get('simulation', {}).get('obj_rate', 3.44) == 1.22:
    coeff = np.polyfit(np.log(LR_range[21:]), np.log(y[21:]), 1)
    #coeff_plot = np.polyfit(np.log(x_idx[21:]), np.log(y[21:]), 1)
else:
    coeff = np.polyfit(np.log(LR_range), np.log(y), 1)
    #coeff_plot = np.polyfit(np.log(x_idx), np.log(y), 1)

print("Fit coefficients (LR, DECAY):", coeff)
np.save(project_root / Path("data") / f"fit_res_{str(config['simulation']['obj_rate']).replace('.','-')}.npy", coeff)
savemat(project_root / Path("data") / f"fit_res_{str(config['simulation']['obj_rate']).replace('.','-')}.mat", {'fit_res': coeff})


Assuming G Axis is the last axis
Fit coefficients (LR, DECAY): [-0.92416023 12.42424635]


In [19]:
min_mm_pos.shape

(100,)

In [9]:
np.argmin(np.abs(hom_grid_mean), axis=0).shape

(110,)

## Save FIC for Mixed model

In [20]:
experiment_manager = ExperimentManager(project_root, config_path="Dynamic_Stability", results_dir="/network/iss/cohen/data/Ivan/fastHDMF")
res_dynamic = experiment_manager.load_experiment_results()
mean_fic_dynamic = res_dynamic['full_results']['observables']['mean_fic']

g_values = res_dynamic['full_results']['axis_values']['G']
for idx, g in enumerate(g_values): 
    avg_fic = np.mean(np.mean(mean_fic_dynamic[idx, :, :],axis=1))
    g_value_str = f"{g:.2f}".replace('.','')
    fname = f"mean_fic_G_{g_value_str}.npy"
    np.save(project_root / "data" / "dyn_fics" / fname, avg_fic)

2025-09-22 16:07:28,999 - hdmf_experiment_Dynamic_Stability - INFO - Experiment 'Dynamic_Stability' initialized
2025-09-22 16:07:29,001 - hdmf_experiment_Dynamic_Stability - INFO - Config: Showing firing rate stability across G values in the dynamic model
2025-09-22 16:07:29,002 - hdmf_experiment_Dynamic_Stability - INFO - Results will be saved to: /network/iss/cohen/data/Ivan/fastHDMF/Dynamic_Stability
2025-09-22 16:07:29,021 - hdmf_experiment_Dynamic_Stability - INFO - Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Dynamic_Stability.yaml


INFO: Loaded config from: /network/iss/home/ivan.mindlin/Repos/fastHDMF/configs/experiments/Dynamic_Stability.yaml
